In [47]:
#Import Libraries
import sqlite3
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

In [38]:
#Sqlite Connection
conn = sqlite3.connect("NBA_db.db")

In [39]:
#getters for DB
def getAllPlayers():
    df = pd.read_sql_query(f"SELECT * from Players",conn)
    return df

def getPlayer(Column,Value):
    df = pd.read_sql_query(f"SELECT * from Players where {Column} = '{Value}'",conn)
    return df

def getBoxScore(Column,Value):
    df1 = pd.read_sql_query(f"SELECT * from BoxScoreBasic where {Column} = '{Value}'",conn)
    df2 = pd.read_sql_query(f"SELECT * from BoxScoreAdvanced where {Column} = '{Value}'",conn)
    return df1,df2

def getPlayByPlay(Token):
    df = pd.read_sql_query(f"SELECT * from PlayByPlay where Token = '{Token}'",conn)
    return df

def getSchedule():
    df = pd.read_sql_query(f"SELECT * from Schedule",conn)
    return df


In [40]:
#Helper Functions

def minutesPlayedHelper(mp):
    mp = mp.split(':')
    return float(mp[0])+float(mp[1])/60

def timeHelper(t,qtr):
    t = t.split(':')
    t = float(t[0])+(float(t[1])/60)
    return ((float(qtr[0])-1)*12)+(12-t)


def playerBoxScoreHelper(Player,Game_Type,column):
    df1,df2 = getBoxScore("Player",Player)
    df = df1 if column in df1.columns.tolist() else df2
    df = df[df.Game_Type == f"{Game_Type}"]
    df = df[df.MP.str.contains(':')]
    if column == "MP":
            df["MP"] = df["MP"].apply(minutesPlayedHelper)
    return df[column]

def playerNameHelper(name):
    name = name.replace(".",".*")
    
    df = getAllPlayers()
    df = df.Player.str.extractall(rf'({name}+ ")')
    if df.shape[0]==0:
        df = getAllPlayers()
        name = name.split()
        name = name[0]+" "+name[1][0:3]+".*"
        
        df = df.Player.str.extractall(rf'({name})')
    return df



In [101]:
#PlayByPlay Functions

def playerIntervals(Token):
    tracker = {}
    lastSeen = {}
    df,_ = getBoxScore("Token",Token)
    df = df[(df.Game_Type == "game") & (df.BenchStatus == "S")]
    for player in df["Player"]:
        tracker[player]=[[0,-1]]
    df = getPlayByPlay(Token)
    df = df[df.AwayTeamPlay.str.contains(" enters the game for ") | df.HomeTeamPlay.str.contains(" enters the game for ") ]

    for play in df.itertuples():
        
        if " enters the game for " in play.AwayTeamPlay :
            subs = play.AwayTeamPlay.split(" enters the game for ")
            subs = [playerNameHelper(s).iloc[0].values[0] for s in subs] 
        else:
            subs = play.HomeTeamPlay.split(" enters the game for ")
            subs = [playerNameHelper(s).iloc[0].values[0] for s in subs] 
        
        t = timeHelper(play.Time,play.Quarter)
        
        if subs[1] not in tracker:
            tracker[subs[1]] = [[timeHelper("12:0",play.Quarter),t]]
        else:
            tracker[subs[1]][-1][1] = t
        if subs[1] in lastSeen:
            del lastSeen[subs[1]]
    
        if subs[0] in tracker:
            if subs[0] in lastSeen:
                tracker[subs[0]][-1][1] = timeHelper("0:0",lastSeen[subs[0]])
            tracker[subs[0]].append([t,-1])
        else:
            tracker[subs[0]] = [[t,-1]]
        lastSeen[subs[0]] = play.Quarter
    
    for k in lastSeen:
        tracker[k][-1][1] = timeHelper("0:0",lastSeen[k])
    return tracker

def playerOnCourt(player, playing_intervals):
    
    result = {}
    player_intervals = playing_intervals.get(player, [])

    for other_player, intervals in playing_intervals.items():
        if other_player == player:
            continue 
   
        overlapping_intervals = []
        for p_start, p_end in player_intervals:
            for o_start, o_end in intervals:
                
                overlap_start = max(p_start, o_start)
                overlap_end = min(p_end, o_end)
                
                if overlap_start < overlap_end:  
                    overlapping_intervals.append([overlap_start, overlap_end])
        
        if overlapping_intervals:
            result[other_player] = overlapping_intervals

    return result

def playOnCourt(player,onCourtPlayer,Token):
    result = playerOnCourt(player,playerIntervals(Token))

    result = result[onCourtPlayer]

    filteredDf = pd.DataFrame()
    df = getPlayByPlay(Token)
    for r in result:
        for play in df.itertuples():
            if r[0]<=timeHelper(play.Time,play.Quarter)<=r[1]:
                filteredDf = pd.concat([filteredDf, pd.DataFrame([play._asdict()])], ignore_index=True)
    return filteredDf




,Index,PlayByPlayID,Quarter,Time,AwayTeamPlay,AwayTeamScoreDiff,Score,HomeTeamScoreDiff,HomeTeamPlay,Token
0,0,1433,1st Q,12:00.0,Jump ball: K. Porzingis vs. M. Robinson (D. Wh...,,,,,202310250NYK
1,1,1434,1st Q,11:48.0,K. Porzingis misses 3-pt jump shot from 23 ft,,0-0,,,202310250NYK
2,2,1435,1st Q,11:46.0,,,0-0,,Defensive rebound by J. Randle,202310250NYK
3,3,1436,1st Q,11:31.0,,,0-0,,J. Brunson misses 2-pt jump shot from 11 ft,202310250NYK
4,4,1437,1st Q,11:29.0,Defensive rebound by K. Porzingis,,0-0,,,202310250NYK
...,...,...,...,...,...,...,...,...,...,...
239,490,1923,4th Q,0:06.0,,,108-104,,J. Brunson misses 3-pt jump shot from 27 ft,202310250NYK
240,491,1924,4th Q,0:01.0,,,108-104,,Offensive rebound by I. Quickley,202310250NYK
241,492,1925,4th Q,0:00.0,,,108-104,,J. Brunson misses 3-pt jump shot from 25 ft,202310250NYK
242,493,1926,4th Q,0:00.0,,,108-104,,Offensive rebound by Team,202310250NYK
